<a href="https://colab.research.google.com/github/MFY10/E-commerce_PredictionML/blob/main/Second_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## E-Commerce Prediction

In [256]:
pip install --upgrade pandas

# Tahap 1 : Pengumpulan Data

Kode ini membagi data menjadi training dan test set, lalu menormalisasi fitur dengan StandardScaler. Selanjutnya, model Logistic Regression dan Random Forest dilatih dan dievaluasi menggunakan classification_report dan confusion_matrix untuk mengukur akurasi, precision, recall, dan F1-score dalam memprediksi kelas.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [3]:
# 1. Load Data
df = pd.read_csv("e-commerce.csv")  # ganti nama file sesuai dataset lo
df.head()

,Customer ID,Gender,Age,City,Membership Type,Total Spend,Items Purchased,Average Rating,Discount Applied,Days Since Last Purchase,Satisfaction Level
0,101,Female,29,New York,Gold,1120.20,14,4.6,True,25,Satisfied
1,102,Male,34,Los Angeles,Silver,780.50,11,4.1,False,18,Neutral
2,103,Female,43,Chicago,Bronze,510.75,9,3.4,True,42,Unsatisfied
3,104,Male,30,San Francisco,Gold,1480.30,19,4.7,False,12,Satisfied
4,105,Male,27,Miami,Silver,720.40,13,4.0,True,55,Unsatisfied


In [4]:
# untuk cek data yang kosong
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer ID               350 non-null    int64  
 1   Gender                    350 non-null    object 
 2   Age                       350 non-null    int64  
 3   City                      350 non-null    object 
 4   Membership Type           350 non-null    object 
 5   Total Spend               350 non-null    float64
 6   Items Purchased           350 non-null    int64  
 7   Average Rating            350 non-null    float64
 8   Discount Applied          350 non-null    bool   
 9   Days Since Last Purchase  350 non-null    int64  
 10  Satisfaction Level        348 non-null    object 
dtypes: bool(1), float64(2), int64(4), object(4)
memory usage: 27.8+ KB


In [5]:
print(df.columns)

Index(['Customer ID', 'Gender', 'Age', 'City', 'Membership Type',
       'Total Spend', 'Items Purchased', 'Average Rating', 'Discount Applied',
       'Days Since Last Purchase', 'Satisfaction Level'],
      dtype='object')


Kode ini mengganti nama kolom di DataFrame df agar lebih konsisten, menggunakan dictionary untuk mencocokkan nama kolom lama dan baru, dengan perubahan diterapkan langsung pada DataFrame.

In [6]:
# Rename Kolom

df.rename(columns={
    'Satisfaction Level': 'Satisfaction_Level',
    'Total Spend': 'Total_Spend',
    'Average Rating': 'Average_Rating',
    'Membership Type': 'Membership_Type',
    'Items Purchased': 'Items_Purchased',
    'Discount Applied': 'Discount_Applied',
    'Customer ID' : 'Customer_ID',
    'Days Since Last Purchase': 'Days_Since_Last_Purchase',
    'City': 'City'
}, inplace=True)

print(df.columns)

Index(['Customer_ID', 'Gender', 'Age', 'City', 'Membership_Type',
       'Total_Spend', 'Items_Purchased', 'Average_Rating', 'Discount_Applied',
       'Days_Since_Last_Purchase', 'Satisfaction_Level'],
      dtype='object')


Fungsi **fill_satisfaction** mengisi nilai kosong pada kolom Satisfaction_Level berdasarkan **Total_Spend** dan **Days_Since_Last_Purchase**. Jika **Total_Spend** kurang dari 1000 dan **Days_Since_Last_Purchase** lebih dari 30, maka diisi dengan 'Unsatisfied'. Jika tidak, diisi 'Neutral' atau 'Satisfied' tergantung Total_Spend. Jika sudah ada nilai, nilai tersebut dibiarkan.

In [7]:
def fill_satisfaction(row):
    if pd.isna(row['Satisfaction_Level']):
        if row['Total_Spend'] < 1000:
            if row['Days_Since_Last_Purchase'] > 30:
                return 'Unsatisfied'
            else:
                return 'Neutral'
        else:
            return 'Satisfied'
    else:
        return row['Satisfaction_Level']

print(df.columns)

Index(['Customer_ID', 'Gender', 'Age', 'City', 'Membership_Type',
       'Total_Spend', 'Items_Purchased', 'Average_Rating', 'Discount_Applied',
       'Days_Since_Last_Purchase', 'Satisfaction_Level'],
      dtype='object')


Kode ini mengisi nilai yang hilang (missing values) di kolom Satisfaction_Level dengan menggunakan fungsi fill_satisfaction yang sudah didefinisikan sebelumnya.

Fungsi apply digunakan untuk menerapkan fill_satisfaction pada setiap baris data (dengan parameter axis=1 untuk mengindikasikan pengolahan per baris). Setelah nilai yang hilang diisi, nama kolom DataFrame ditampilkan dengan df.columns.

In [8]:
# Handle missing values
df['Satisfaction_Level'] = df.apply(fill_satisfaction, axis=1)

print(df.columns)

Index(['Customer_ID', 'Gender', 'Age', 'City', 'Membership_Type',
       'Total_Spend', 'Items_Purchased', 'Average_Rating', 'Discount_Applied',
       'Days_Since_Last_Purchase', 'Satisfaction_Level'],
      dtype='object')


In [9]:
print(df.dtypes)

Customer_ID                   int64
Gender                       object
Age                           int64
City                         object
Membership_Type              object
Total_Spend                 float64
Items_Purchased               int64
Average_Rating              float64
Discount_Applied               bool
Days_Since_Last_Purchase      int64
Satisfaction_Level           object
dtype: object


Kode ini menampilkan 5 nilai pertama dari kolom City, Membership_Type, dan Satisfaction_Level menggunakan metode head() untuk memeriksa data.

In [10]:
print(df['City'].head())  # Menampilkan 5 nilai pertama dari kolom 'City'
print(df['Membership_Type'].head())  # Menampilkan 5 nilai pertama dari kolom 'Membership_Type'
print(df['Satisfaction_Level'].head())  # Menampilkan 5 nilai pertama dari kolom 'Satisfaction_Level'

0         New York
1      Los Angeles
2          Chicago
3    San Francisco
4            Miami
Name: City, dtype: object
0      Gold
1    Silver
2    Bronze
3      Gold
4    Silver
Name: Membership_Type, dtype: object
0      Satisfied
1        Neutral
2    Unsatisfied
3      Satisfied
4    Unsatisfied
Name: Satisfaction_Level, dtype: object


Kode ini mengubah tipe data kolom Membership_Type, Satisfaction_Level, dan City menjadi string.

In [11]:
df['Membership_Type'] = df['Membership_Type'].astype('str')
df['Satisfaction_Level'] = df['Satisfaction_Level'].astype('str')
df['City'] = df['City'].astype('str')

Kode ini mengubah nilai kategorikal dalam DataFrame menjadi numerik: Gender (Male=1, Female=0), City (berdasarkan urutan kota), Membership_Type (Bronze=0, Silver=1, Gold=2), Satisfaction_Level (Unsatisfied=0, Neutral=1, Satisfied=2), dan Discount_Applied (True=1, False=0). Ini mempersiapkan data untuk model machine learning.

In [13]:
#Encoding

df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})

city_order = {
    'Houston': 0,
    'Los Angeles': 1,
    'New York': 2,
    'San Francisco': 3,
    'Chicago': 4,
    'Miami': 5,
    'Boston': 6,
    'Seattle': 7,
    'Dallas': 8
}

# Menerapkan mapping ke kolom 'City'
df['City'] = df['City'].map(city_order)

membership_order = {
    'Bronze': 0,
    'Silver': 1,
    'Gold': 2
}

df['Membership_Type'] = df['Membership_Type'].map(membership_order)

satisfaction_order = {
    'Unsatisfied': 0,
    'Neutral': 1,
    'Satisfied': 2
}

df['Satisfaction_Level'] = df['Satisfaction_Level'].map(satisfaction_order)
df['Discount_Applied'] = df['Discount_Applied'].map({True: 1, False: 0})

df

,Customer_ID,Gender,Age,City,Membership_Type,Total_Spend,Items_Purchased,Average_Rating,Discount_Applied,Days_Since_Last_Purchase,Satisfaction_Level
0,101,0,29,2,2,1120.20,14,4.6,1,25,2
1,102,1,34,1,1,780.50,11,4.1,0,18,1
2,103,0,43,4,0,510.75,9,3.4,1,42,0
3,104,1,30,3,2,1480.30,19,4.7,0,12,2
4,105,1,27,5,1,720.40,13,4.0,1,55,0
...,...,...,...,...,...,...,...,...,...,...,...
345,446,1,32,5,1,660.30,10,3.8,1,42,0
346,447,0,36,0,0,470.50,8,3.0,0,27,1
347,448,0,30,2,2,1190.80,16,4.5,1,28,2
348,449,1,34,1,1,780.20,11,4.2,0,21,1


Kode ini menormalisasi fitur numerik (Age, Total_Spend, Items_Purchased, Days_Since_Last_Purchase) dengan StandardScaler dan memisahkan data menjadi X (fitur) dan y (target: Satisfaction_Level) untuk persiapan model machine learning.

In [17]:
# Normalisasi fitur numerik
scaler = StandardScaler()
df[['Age', 'Total_Spend', 'Items_Purchased', 'Days_Since_Last_Purchase']] = scaler.fit_transform(df[['Age', 'Total_Spend', 'Items_Purchased', 'Days_Since_Last_Purchase']])

X = df.drop('Satisfaction_Level', axis=1)  # Fitur (semua kolom selain 'Satisfaction_Level')
y = df['Satisfaction_Level']  # Target


Kode ini membagi data menjadi training set (80%) dan test set (20%) menggunakan train_test_split dari scikit-learn, dengan random_state=42 untuk memastikan hasil yang konsisten. X_train dan y_train digunakan untuk melatih model, sementara X_test dan y_test digunakan untuk menguji kinerja model.

In [20]:
# 3. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Kode ini membuat dan melatih model Logistic Regression dengan parameter max_iter=1000 untuk memastikan konvergensi, serta random_state=42 untuk konsistensi. Model dilatih menggunakan X_train dan y_train sebagai data input dan target.

In [24]:
# 4. Modeling – Logistic Regression (Baseline)
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

Kode ini memprediksi Satisfaction_Level pada data X_test, lalu mengevaluasi model dengan menampilkan matriks kebingungannya dan laporan klasifikasi yang mencakup metrik seperti precision, recall, dan f1-score untuk menilai kinerja model.

In [23]:
# 5. Evaluate
y_pred = model.predict(X_test)

print("=== Logistic Regression ===")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

=== Logistic Regression ===
[[18  0  0]
 [ 0 23  0]
 [ 0  1 28]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       0.96      1.00      0.98        23
           2       1.00      0.97      0.98        29

    accuracy                           0.99        70
   macro avg       0.99      0.99      0.99        70
weighted avg       0.99      0.99      0.99        70



Kode ini melatih model Random Forest menggunakan X_train dan y_train, kemudian memprediksi Satisfaction_Level pada X_test dengan model yang telah dilatih.

In [25]:
# 6. Bonus: Coba RandomForest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

In [26]:
print("=== Random Forest ===")
print(confusion_matrix(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

=== Random Forest ===
[[18  0  0]
 [ 0 23  0]
 [ 0  0 29]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       1.00      1.00      1.00        23
           2       1.00      1.00      1.00        29

    accuracy                           1.00        70
   macro avg       1.00      1.00      1.00        70
weighted avg       1.00      1.00      1.00        70



Project ini berpotensi mengalami overfitting karena data yang digunakan hampir tidak memiliki nilai yang hilang (missing values). Model mungkin terlalu cocok dengan data pelatihan, sehingga kinerjanya bisa menurun saat diterapkan pada data yang tidak terlihat sebelumnya. Sebaiknya dilakukan validasi lebih lanjut untuk memastikan model dapat generalisasi dengan baik.